In [34]:
# native packs
import requests
import json
# installed packs
import pandas as pd
from pandas import json_normalize


auth_endpoint = "https://auth.emsicloud.com/connect/token"  # auth endpoint

# replace 'your_client_id' with your client id from your api invite email
client_id = "5f379hywuvh7fvan"
# replace 'your_client_secret' with your client secret from your api invite email
client_secret = "hfCkXQEy"
scope = "emsi_open"  # ok to leave as is, this is the scope we will used

# set credentials and scope
payload = f"client_id={client_id}&client_secret={client_secret}&grant_type=client_credentials&scope={scope}"
# headers for the response
headers = {'content-type': 'application/x-www-form-urlencoded'}
access_token = json.loads((requests.request("POST", auth_endpoint, data=payload, headers=headers)).text)[
    'access_token']  # grabs request's text and loads as JSON, then pulls the access token from that

fields = [
      "id",
      "type",
      "name",
      "isSoftware",
      "infoUrl",
      "tags",
      "isLanguage",
      "description",
      "descriptionSource",
      "category",
      "subcategory"
    ]
fields_str = ','.join(fields)

querystring = { "fields":fields_str}

def fetch_skills_list() -> pd.DataFrame:

    # List of all skills endpoint
    all_skills_endpoint = "https://emsiservices.com/skills/versions/latest/skills"
    # Auth string including access token from above
    auth = f"Authorization: Bearer {access_token}"
    headers = {'authorization': auth}  # headers
    response = requests.request(
        "GET", all_skills_endpoint, headers=headers,params = querystring)  # response
    response = response.json()['data']  # the data

    # all_skills_df = pd.DataFrame(json_normalize(response)); # Where response is a JSON object drilled down to the level of 'data' key
    return response

In [35]:
response = fetch_skills_list()

In [36]:
all_skills_df = pd.DataFrame(json_normalize(response))

In [46]:
all_skills_df[all_skills_df['isLanguage']==True]

,description,descriptionSource,id,infoUrl,isLanguage,isSoftware,name,tags,category.id,category.name,subcategory.id,subcategory.name,type.id,type.name,category,subcategory
941,Afrikaans is a West Germanic language spoken i...,https://en.wikipedia.org/wiki/Afrikaans,ESBBBBACBA39B30028C8,https://skills.emsidata.com/skills/ESBBBBACBA3...,True,False,Afrikaans Language,"[{'key': 'wikipediaExtract', 'value': 'Afrikaa...",23.0,Media and Communications,498.0,Language Competency,ST1,Specialized Skill,NaN,NaN
1007,"The Ainu languages, sometimes also known as Ai...",https://en.wikipedia.org/wiki/Ainu_languages,KS7G4BZ6YYJX84KS61V3,https://skills.emsidata.com/skills/KS7G4BZ6YYJ...,True,False,Ainu (Language),"[{'key': 'wikipediaExtract', 'value': 'The Ain...",0.0,NULL,100.0,NULL,ST1,Specialized Skill,NaN,NaN
1097,Akan is a Central Tano language and the princi...,https://en.wikipedia.org/wiki/Akan_language,KS7G3LS68MJ3FTGCYPP0,https://skills.emsidata.com/skills/KS7G3LS68MJ...,True,False,Akan (Language),"[{'key': 'wikipediaExtract', 'value': 'Akan is...",23.0,Media and Communications,498.0,Language Competency,ST1,Specialized Skill,NaN,NaN
1100,Akkadian is an extinct East Semitic language t...,https://en.wikipedia.org/wiki/Akkadian_language,KS120CY70MMPBDBHPJ7M,https://skills.emsidata.com/skills/KS120CY70MM...,True,False,Akkadian Language,"[{'key': 'wikipediaExtract', 'value': 'Akkadia...",0.0,NULL,100.0,NULL,ST1,Specialized Skill,NaN,NaN
1106,Albanian is an Indo-European language spoken b...,https://en.wikipedia.org/wiki/Albanian_language,ES33595A31E4D47C9F2A,https://skills.emsidata.com/skills/ES33595A31E...,True,False,Albanian Language,"[{'key': 'wikipediaExtract', 'value': 'Albania...",23.0,Media and Communications,498.0,Language Competency,ST1,Specialized Skill,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31386,"Wolof is a language of Senegal, the Gambia and...",https://en.wikipedia.org/wiki/Wolof_language,KS4424L6SMXXJPTXQCCX,https://skills.emsidata.com/skills/KS4424L6SMX...,True,False,Wolof Languages,"[{'key': 'wikipediaExtract', 'value': 'Wolof i...",23.0,Media and Communications,498.0,Language Competency,ST1,Specialized Skill,NaN,NaN
31504,None,None,KS4425C6ZM53B0RZCD8Y,https://skills.emsidata.com/skills/KS4425C6ZM5...,True,False,Written English,"[{'key': 'wikipediaExtract', 'value': 'A writt...",23.0,Media and Communications,567.0,Writing and Editing,ST2,Common Skill,NaN,NaN
31675,"Xhosa also isiXhosa as an endonym, is a Nguni ...",https://en.wikipedia.org/wiki/Xhosa_language,KS7G6YW6YNMG2886GN10,https://skills.emsidata.com/skills/KS7G6YW6YNM...,True,False,Xhosa (Language),"[{'key': 'wikipediaExtract', 'value': 'Xhosa a...",23.0,Media and Communications,498.0,Language Competency,ST1,Specialized Skill,NaN,NaN
31730,Yiddish is a Hebrew-High German language histo...,https://en.wikipedia.org/wiki/Yiddish_Language,KS4428P6CJB8SVNNV60T,https://skills.emsidata.com/skills/KS4428P6CJB...,True,False,Yiddish Language,"[{'key': 'wikipediaExtract', 'value': 'Yiddish...",23.0,Media and Communications,498.0,Language Competency,ST1,Specialized Skill,NaN,NaN


In [37]:
all_skills_df.groupby('category.name').count()

,description,descriptionSource,id,infoUrl,isLanguage,isSoftware,name,tags,category.id,subcategory.id,subcategory.name,type.id,type.name,category,subcategory
category.name,,,,,,,,,,,,,,,
Administration,159,159,218,218,218,218,218,218,218,218,218,218,218,0,0
"Agriculture, Horticulture, and Landscaping",95,95,113,113,113,113,113,113,113,113,113,113,113,0,0
Analysis,558,558,703,703,703,703,703,703,703,703,703,703,703,0,0
Architecture and Construction,225,225,361,361,361,361,361,361,361,361,361,361,361,0,0
Business,633,633,965,965,965,965,965,965,965,965,965,965,965,0,0
Customer and Client Support,52,52,95,95,95,95,95,95,95,95,95,95,95,0,0
Design,356,356,495,495,495,495,495,495,495,495,495,495,495,0,0
"Economics, Policy, and Social Studies",203,203,243,243,243,243,243,243,243,243,243,243,243,0,0
Education and Training,273,273,389,389,389,389,389,389,389,389,389,389,389,0,0


In [42]:
all_skills_df[all_skills_df['category.name']=='Administration'][['id','name','isLanguage','isSoftware','category.name','subcategory.name','description']]

,id,name,isLanguage,isSoftware,category.name,subcategory.name,description
334,KS1228L6JG2Z7Y9388W1,Abstract Management,False,False,Administration,Document Management,Abstract management is the process of acceptin...
615,BGS4C2319B8365DF8423,Adding Machines,False,False,Administration,Office and Productivity Equipment and Technology,An adding machine is a class of mechanical cal...
649,ESE1B279F38C73E025BD,Administrative Functions,False,False,Administration,Administrative Support and Clerical Tasks,A person responsible for providing various kin...
653,ES2F712D74818EF2888E,Administrative Support,False,False,Administration,Administrative Support and Clerical Tasks,A person responsible for providing various kin...
656,KSEJWTKAA1P00B63W2BR,Admissions Operations,False,False,Administration,Administrative Support and Clerical Tasks,None
...,...,...,...,...,...,...,...
31026,KS4420B71HQPGBB6QS0T,Webmail,False,True,Administration,Office and Productivity Equipment and Technology,\nWebmail is an email service that can be acce...
31406,KS1233L6BGZ6R7D2FX0H,Word Processing,False,False,Administration,Administrative Support and Clerical Tasks,A word processor (WP) is a device or computer ...
31668,BGS937CBD2778845F9D9,Xerox Copiers,False,False,Administration,Office and Productivity Equipment and Technology,A photocopier is a machine that makes copies o...
31669,KS1233R6Y1G45GN4H35D,Xerox DocuShare,False,True,Administration,Document Management,DocuShare is a content management system devel...
